Code created following tutorial: https://thepythoncodingbook.com/2021/12/11/simulating-3d-solar-system-python-matplotlib/ 

In [15]:
import math
import matplotlib.pyplot as plt
import itertools

Define vectors as a class. The __init__ method initializes attributes of the vector object as soon as the object is formed. The rest of the class defines different mathematical operations with vectors as well as finding the magnitude of a vector and normalizing it.

In [16]:
class Vector: 
    
    def __init__(self, x=0.0, y=0.0, z=0.0): # if not defined, x,y,z default value is 0. Add decimal to avoid warnings when dividing self.x etc.
        self.x = x
        self.y = y
        self.z = z

    def __str__(self): # returns the vector in usual coordinates
        return f" ({self.x}, {self.y}, {self.z}) "

    def __getitem__(self, item): # makes class indexable, so vector becomes a list
        if item == 0:
            return self.x
        elif item == 1:
            return self.y
        elif item == 2:
            return self.z
        else:
            raise IndexError("There are only three elements in the vector")
    
    def __add__(self, other): # adding vectors looks nicer this way - adding vectors in python needs numpy functions
        return Vector(
            self.x + other.x,
            self.y + other.y,
            self.z + other.z
        )
    
    def __sub__(self, other):
        return Vector(
            self.x - other.x,
            self.y - other.y,
            self.z - other.z
        )
    
    def __mul__(self, other): # defining multiplication with a vector
        if isinstance(other, Vector): # vector scalar product (if first argument is of type second argument)
            return(
                self.x * other.x + self.y * other.y + self.z * other.z
            )
        elif isinstance(other, (int, float)): # vector multiplied by scalar - in the multiplication, vector is first and scalar second
            return Vector(
                self.x * other, self.y * other, self.z * other
            )
        else:
            raise TypeError("Operant must be vector, int og float") # stops the code and tells us that there is an error
        
    def __truediv__(self, other): # defining vector divided by scalar
        if isinstance(other, (int, float)):
            return Vector(
                self.x / other, self.y / other, self.z / other
            )
        else: 
            raise TypeError("Operand must be int or float")
    
    def length(self): # the length of the vector
        return math.sqrt( self.x**2 + self.y**2 + self.z**2 )

    def normalize(self): # normalize vector, i.e. length 1 but preserve direction
        length = self.length()
        return Vector(
            self.x / length, self.y / length, self.z / length
        )

Define the SolarSystem class that will take care of the solar systen and keep track of how many bodies it contains and the interactions betweeen them.

In [17]:
class SolarSystem: 
    def __init__(self, size, projection_2d=False): # last argument allows to toggle between visualisation options. If false, only shows in 3D, if true also shows 2D on the 'floor' of the animation.
        self.projection_2d = projection_2d
        self.size = size # size of cube that contains solar system
        self.bodies = [] # will contain all bodies in the solar system

        self.fig, self.ax = plt.subplots( # create figure and set of axes
            1, # first two arguments are 1 to create single set of axes in figure
            1,
            subplot_kw = {"projection": "3d"}, # has dictionary as argument, sets projection to 3D. I.e. axes created are an Axes3D object
            figsize = (self.size / 50, self.size / 50) # sets overall size of figure containing the Axs3D object.
        )
        self.fig.tight_layout() # reduces margins at edge of figure
        if self.projection_2d:
            self.ax.view_init(10,0) # elevation set to 10 degrees when 2D projection is on to allow bottom plane to be visible.
        else: 
            self.ax.view_init(0, 0) # set azimuth and elevation of view to 0, so x-axis is perp. to screen

    def add_body(self, body): # add orbiting bodies to the solar system
        self.bodies.append(body)
    
    def update_all(self): # move each body and draw them at the new position
        self.bodies.sort(key=lambda item: item.position[0]) # sorts bodies based on x-coordinate each time the plot is refreshed so that the ones furthest back are first so that it looks like they are behind those in front
        for body in self.bodies:                            # lambda function is small anonymous function. can be used as key function when sorting a list. 
            body.move()
            body.draw()

    def draw_all(self): # sets axes limits and updates the plot.
        self.ax.set_xlim( (-self.size/2 , self.size/2) ) #limit of three axes using solar system's size
        self.ax.set_ylim( (-self.size/2 , self.size/2) )
        self.ax.set_zlim( (-self.size/2 , self.size/2) )
        if self.projection_2d: # keep grid and axes but remove labels on the axes
            self.ax.xaxis.set_ticklabels([])
            self.ax.yaxis.set_ticklabels([])
            self.ax.zaxis.set_ticklabels([])
        else:
            self.ax.axis(False) # use to remove axes and grid
        plt.pause(0.001) # updates the plot
        self.ax.clear() # clears axes, ready for next plot

    def calculate_all_body_interactions(self): # work out interaction between all bodies in solar system
        bodies_copy = self.bodies.copy() # use a copy to care for possibility that bodies will be removed from solar system during loop - might happen in some cases.
        for idx, first in enumerate(bodies_copy): # enumerate gives 2 loop variables: count of current iteration and the value of the item at current iteration
            for second in bodies_copy[idx+1:]: # make sure interaction between the same two bodies is not calculated twice by only calculating for a body and the bodies following that in the list
                first.acceleration_due_to_gravity(second) # calculate the acceleration of the first body due to the second


Define the SolarSystemBody class that has to do with the individual bodies in the solar system and their movements.
The parameters in the class are:
- solar_system: links the body to a solar system - should be of type SolarSystem
- mass: integer or float that defines the mass of the body.
- position: point in 3D. Default is origin.
- velocity: as input it is a tuple, but is converted into a Vector object.

We will be working with arbitrary units - so each time unit will be one iteration of the loop we'll use to run the simulation, i.e. the move() method will move the body by an amount required for one itereation which is one time unit.

In [18]:
class SolarSystemBody: 
    # these two set up the parameters for determining the size o the markers that will be displayed on the 3D plot.
    min_display_size = 10 # set minimum size so that marker displayed is not too small, even for small bodies.
    display_log_base = 1.3 # use log-scale to convert from mass to marker size
    dt = 100 # time step for numerical integration (euler method)
    
    def __init__(self, solar_system, mass, position=(0,0,0), velocity=(0,0,0)):
        self.solar_system = solar_system 
        self.mass = mass
        self.position = Vector(*position)#position # still a tuple
        self.velocity = Vector(*velocity) # converted to Vector object. The * unpacks the arguments to x,y,x instead of a tuple (x,y,z)

        self.display_size = max( # size and colour of the marker that will be drawn to represent the body
            math.log(self.mass, self.display_log_base), # scale the marker with log(base1.3) of body's mass
            self.min_display_size
            )
        self.colour = "black" # colour of marker

        self.solar_system.add_body(self) # add this body to the solar system
    
    def acceleration_due_to_gravity(self, other): # calculate gravitational force between two bodies and changes velocities of them
        distance = Vector(*other.position) - Vector(*self.position)
        distance_mag = distance.length()

        force = distance.normalize() * self.mass * other.mass / (distance_mag**2) # arbitrary units so gravitational constant omitted
        #force_mag = force.length()

        reverse = 1 # on the body itself the acceleration is given as calculated - but on the other body it is opposite (Newton 3)
        for body in self, other:
            acceleration = force / body.mass
            body.velocity += acceleration * reverse # new velocity value after one time unit
            reverse = -1 # on the other body the acceleration has opposite sign - the loop is only over the two bodies
    
    def move(self):
        self.position = ( # working with arbitrary units, so one iteration corresponds to one time unit.
            self.position + self.velocity
            #self.position[0] + self.velocity[0],
            #self.position[1] + self.velocity[1],
            #self.position[2] + self.velocity[2]
        )
    
    def draw(self):
        self.solar_system.ax.plot(
            *self.position,#self.position.x, self.position.y, self.position.z, #*self.position
            marker="o",
            #markersize=self.display_size, #
            markersize=self.display_size + self.position[0] / 30, # can improve 3D visualization by changing size of marker depending on its x-coordinates. Objects closer appear larger etc.
            color=self.colour
        )
        if self.solar_system.projection_2d: # add 2D projection
            self.solar_system.ax.plot(
                self.position.x,
                self.position.y, # plot x- and y-coordinates
                -self.solar_system.size / 2, # instead of z-coordinate use minimum value of z representing the 'floor'
                marker="o", 
                markersize = self.display_size/2,
                color = (0.5, 0.5, 0.5) # plot grey marker half the size of the main markers in the animation
            )

Now focus on two types of bodies: suns and planets. Two classes are created for these which inherit from SolarSystemBody

In [19]:
class Sun(SolarSystemBody):
    def __init__(self, solar_system, mass=10_000, position=(0,0,0), velocity=(0,0,0)):
        super(Sun,self).__init__(solar_system, mass, position, velocity) # super function refers to parent class or superclass. Allows to call methods defined in superclass from this subclass
        self.colour = "yellow"

class Planet(SolarSystemBody):
    colours = itertools.cycle( [(1,0,0), (0,1,0), (0,0,1)] ) # (red, green, blue)
    def __init__(self, solar_system, mass=10, position=(0,0,0), velocity=(0,0,0)):
        super(Planet,self).__init__(solar_system, mass, position, velocity)
        self.colour = next(Planet.colours) # cycle through RGB colours

In [21]:
# plot figure in a window too instead of only below so that animation is shown
%matplotlib auto

solar_system = SolarSystem(400, projection_2d=True)

#sun = Sun(solar_system, mass=1.9885*10**30) #mass in kg
#earth = Planet(solar_system, mass=5.972168*10**24, )

sun = Sun(solar_system)
planets = (Planet(solar_system, position=(150, 50, 0), velocity=(0, 5, 5)),
    Planet(solar_system, mass=20, position=(100, -50, 150), velocity=(5, 0, 0)))

while True:
    solar_system.calculate_all_body_interactions()
    solar_system.update_all()
    solar_system.draw_all()

Using matplotlib backend: <object object at 0x0000014B81EFB380>


KeyboardInterrupt: 